In [9]:
import numpy as np
import scipy.stats as st
import cvxpy as cvx
import pandas as pd
import datetime

In [10]:
tickers=pd.read_csv('tickers.csv')
tickers=tickers['ticker'].to_list()
data = pd.read_csv('./data/'+tickers[0]+'.csv')
data = data.drop(['Open', 'High','Low','Close','Adj Close','Volume'], axis=1)
for ticker in tickers:
    d = pd.read_csv('./data/'+ticker+'.csv')
    d = d.drop(['Open', 'High','Low','Adj Close','Volume'], axis=1)
    d = d.rename(columns={'Close': ticker})
    data = pd.merge(data,d,on='Date')

In [11]:
cal = data[['Date']]
cal['Datetime'] = pd.to_datetime(cal['Date'])
cal['year'] = cal['Datetime'].dt.year
cal['month'] = cal['Datetime'].dt.month
cal['week'] = cal['Datetime'].dt.week

/tmp/ipykernel_2116/3635024390.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal['Datetime'] = pd.to_datetime(cal['Date'])
/tmp/ipykernel_2116/3635024390.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal['year'] = cal['Datetime'].dt.year
/tmp/ipykernel_2116/3635024390.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [12]:
cal['mon'] = 1
for index, row in cal.iterrows():
    if index > 0 and cal.iloc[index,3] != cal.iloc[index-1,3]:
        cal.iloc[index,5] = cal.iloc[index-1,5]+1
    else:
        cal.iloc[index,5] = cal.iloc[index-1,5]

/tmp/ipykernel_2116/501106492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cal['mon'] = 1


In [13]:
data = data.set_index('Date')

In [14]:
R = data.pct_change()
R = R.dropna()

In [16]:
Benchmark = pd.read_csv('sp500.csv')
Benchmark = Benchmark.set_index('Date')
Benchmark = Benchmark.drop(['Open', 'High','Low','Adj Close','Volume'], axis=1)
Benchmark = Benchmark.pct_change()
Benchmark = Benchmark.dropna()

In [17]:
T = R.shape[0]
N = R.shape[1]
asset_sqt = R/np.sqrt(T)
index_sqt = Benchmark/np.sqrt(T)

In [33]:
di = pd.DataFrame(index=Benchmark.index,columns=['Close'])

In [39]:
w=weight.value
a=asset*w
print(a.shape)

(756, 50)


In [30]:
mon_window = 36
weight = cvx.Variable(N)
for i in range(120 - mon_window+1):
    start = i+1
    end = i+mon_window
    startdate = cal.query('mon >= '+str(start)+' and mon <= '+str(end))['Date'].min()
    enddate = cal.query('mon >= '+str(start)+' and mon <= '+str(end))['Date'].max()
    asset = asset_sqt[startdate:enddate].values
    indx = index_sqt[startdate:enddate].values
    indx = np.squeeze(indx)
    obj = cvx.Minimize(cvx.sum_squares(asset@weight - indx))
    mintracerr = cvx.Problem(obj, [cvx.sum(weight) == 1.0,weight >= .0])
    mintracerr.solve(verbose=True)
    

                                     CVXPY                                     
                                     v1.3.0                                    
(CVXPY) Feb 20 12:59:26 AM: Your problem has 50 variables, 2 constraints, and 0 parameters.
(CVXPY) Feb 20 12:59:26 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 20 12:59:26 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 20 12:59:26 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 20 12:59:26 AM: Compiling problem (target solver=OSQP).
(CVXPY) Feb 20 12:59:26 AM: Reduction chain: CvxAttr2Constr -> Qp2SymbolicQp -> QpMatrixStuffing